In [1]:
!pip install openai --quiet


[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [11]:
import openai

with open("api_key", "r") as f:
    openai.api_key = f.read()

USE_GPT4 = False

In [9]:
class Conversation:
    """
    This class helps me keep the context of a conversation. It's not
    sophisticated at all and it simply regulates the number of messages in the
    context window.

    You could try something much more involved, like counting the number of
    tokens and limiting. Even better: you could use the API to summarize the
    context and reduce its length.

    But this is simple enough and works well for what I need.
    """

    messages = None

    def __init__(self):
        # Here is where you can add some personality to your assistant, or
        # play with different prompting techniques to improve your results.
        Conversation.messages = [
            {
                "role": "system",
                "content": (
                    "You are a helpful, and concise assistant."
                ),
            }
        ]


    def answer(self, prompt):
        """
        This is the function I use to ask questions.
        """
        self._update("user", prompt)

        response = openai.ChatCompletion.create(
            model="gpt-4" if USE_GPT4 else "gpt-3.5-turbo-0613",
            messages=Conversation.messages,
            temperature=0,
        )

        self._update("assistant", response.choices[0].message.content)

        return response.choices[0].message.content

    def _update(self, role, content):
        Conversation.messages.append({
            "role": role,
            "content": content,
        })

        # This is a rough way to keep the context size manageable.
        if len(Conversation.messages) > 20:
            Conversation.messages.pop(0)

In [7]:
prompt = """
How can I change the log level of a Flask application at runtime?
"""

In [13]:
conversation = Conversation()

print(conversation.answer(prompt))

To change the log level of a Flask application at runtime, you can use the `app.logger` object provided by Flask. Here's how you can do it:

1. Import the necessary modules:
```python
from flask import Flask
import logging
```

2. Create a Flask application:
```python
app = Flask(__name__)
```

3. Set the log level:
```python
app.logger.setLevel(logging.DEBUG)
```
Replace `logging.DEBUG` with the desired log level, such as `logging.INFO`, `logging.WARNING`, `logging.ERROR`, etc.

4. Run the Flask application:
```python
if __name__ == '__main__':
    app.run()
```

Now, when you run your Flask application, it will log messages according to the set log level. You can change the log level at any time during runtime by calling `app.logger.setLevel()` with the desired log level.


In [17]:
prompt = """
But the problem is that the Flask app is already running"""

print(conversation.answer(prompt))

If your Flask app is already running and you want to change the log level from ERROR to DEBUG while it is running, you can use the `app.logger.setLevel()` method to update the log level dynamically.

Here's an example of how you can achieve this:

1. Import the necessary modules:
```python
from flask import Flask
import logging
```

2. Create a Flask application:
```python
app = Flask(__name__)
```

3. Set the initial log level to ERROR:
```python
app.logger.setLevel(logging.ERROR)
```

4. Define a route to change the log level:
```python
@app.route('/log-level/<level>')
def set_log_level(level):
    if level == 'debug':
        app.logger.setLevel(logging.DEBUG)
    elif level == 'info':
        app.logger.setLevel(logging.INFO)
    elif level == 'warning':
        app.logger.setLevel(logging.WARNING)
    elif level == 'error':
        app.logger.setLevel(logging.ERROR)
    return f'Log level set to {level.upper()}'
```

5. Run the Flask application:
```python
if __name__ == '__main__